In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
from dateutil.relativedelta import relativedelta
from skinet.utils.athena_tools import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec 
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothicCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [3]:
ym = '202011'                          # 전월 (몇월의 마감자료로 작업하고 있는지)
final_call_boundary = '2020/12/31'     # 최종 통화일 기준 (이 이후 통화건이 있으면 제외)

In [4]:
# 6개월 전 연월일
ymd_minus_6month = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(months=6)
ymd_minus_6month = str(ymd_minus_6month.year)+str(ymd_minus_6month.month).zfill(2)+str(ymd_minus_6month.day).zfill(2)
ymd_minus_6month

'20200501'

In [5]:
# 6개월 전 연월일 (슬래쉬 형식)
ymd_minus_6month_slash = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(months=6)
ymd_minus_6month_slash = str(ymd_minus_6month_slash.year)+'/'+str(ymd_minus_6month_slash.month).zfill(2)+'/'+str(ymd_minus_6month_slash.day).zfill(2)
ymd_minus_6month_slash

'2020/05/01'

In [6]:
# 1년 전 연월일
ymd_minus_1year = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(years=1)
ymd_minus_1year = str(ymd_minus_1year.year)+str(ymd_minus_1year.month).zfill(2)+str(ymd_minus_1year.day).zfill(2)
ymd_minus_1year

'20191101'

## 1. 월 마감자료 만들기

In [7]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.monthly_deadline_{}'.format(ym))
magam = load_from_athena(conn, '''
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_date
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.kunnr as kunnr_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
     , c.stcd2
     , c.j_1kftbus AS uptae
     , c.j_1kftind AS jongmok
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aezet
     , b.aenam
     , e.ctrt_id AS nsok
FROM sap.zsdt4013      AS a 
LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, aezet FROM sap.zsdt3748 WHERE usest = 'X') AS b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1, j_1kftbus, j_1kftind FROM sap.kna1 WHERE mandt = '100') AS c
      ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
LEFT JOIN (select distinct ctrt_id, ctnum from telepro.tb_nsok_mapping) e --n사 고객 추출
       ON a.ctnum = e.ctnum
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2')
'''.format(ym)) 
load_to_athena(conn, magam, 'default.monthly_deadline_{}'.format(ym))

<< Delete s3://adt-sandbox/_create/tables/default.monthly_deadline_202011
...tmp_db name = tmp_20201208_13_22_57_358bc07aadc2640f72e3ded3c4fb03779b946ddc
Start creating table
drop table if exists default.tmp_20201208_13_22_57_358bc07aadc2640f72e3ded3c4fb03779b946ddc
<< table not exist

        create  table         default.tmp_20201208_13_22_57_358bc07aadc2640f72e3ded3c4fb03779b946ddc
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_date
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.kunnr as kunnr_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , c.name1
     , a.gad_add01
  

0

## 2. 기 가입고객 테이블

In [8]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.tmupsell_exclude')
temp = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_nm FROM mart.zsdr6051) c
       ON b.ctnum = c.ctnum
      AND b.rqnum = c.rqnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_old FROM mart.zsdr6001 WHERE ctnum_old <> '') d
       ON b.ctnum = d.ctnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_hash FROM adt_work.hash_info) e
       ON b.ctnum = e.ctnum_hash
LEFT JOIN (SELECT ctnum, gad_start FROM sap.zsdt4013 WHERE stmon = \'{}\') f
       ON b.ctnum = f.ctnum
)

, t2 AS (
SELECT * , 'case1' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '02' AND ins_amt_type >= '20' UNION ALL
SELECT * , 'case2' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '03' AND ins_amt_type >= '50' UNION ALL
SELECT * , 'case3' AS info FROM t1 WHERE ins_stat = '1' AND ins_type IN ('05', '07') UNION ALL
SELECT * , 'case4' AS info FROM t1 WHERE ins_stat = '1' AND ins_user4 = 'TM' UNION ALL
SELECT * , 'case5' AS info FROM t1 WHERE ins_stat = '1' AND valid_sdate >= \'{}\' AND (ctnum_old = '' OR (ctnum_old <> '' AND valid_sdate <> gad_start)) UNION ALL
SELECT * , 'case6' AS info FROM t1 WHERE ins_stat = '2' AND ins_type IN ('02', '03', '07') AND valid_edate >= \'{}\' UNION ALL
SELECT * , 'case7' AS info FROM t1 WHERE ins_stat = '2' AND ins_user4 = 'TM' AND ins_type IN ('05') AND valid_edate >= \'{}\'
)

SELECT DISTINCT *
  FROM t2
 WHERE valid_sdate <> '99991231'
'''.format(ym, ymd_minus_6month, ymd_minus_6month, ymd_minus_6month))
load_to_athena(conn, temp, 'default.tmupsell_exclude'.format(ym))

<< Delete s3://adt-sandbox/_create/tables/default.tmupsell_exclude
...tmp_db name = tmp_20201208_13_25_40_2ff1c76f12af54c3c6be349fff21a93ab09b6fd0
Start creating table
drop table if exists default.tmp_20201208_13_25_40_2ff1c76f12af54c3c6be349fff21a93ab09b6fd0
<< table not exist

        create  table         default.tmp_20201208_13_25_40_2ff1c76f12af54c3c6be349fff21a93ab09b6fd0
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
L

0

In [9]:
del temp

## 3. TM 실적 테이블 (계약번호, 회차, 최종접촉일, 최종접촉결과)

In [10]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.tm_upsell_result_table')
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2_temp AS (
SELECT DISTINCT kunnr, resultcd, dialdate
  FROM t1
WHERE resultcd IS NOT NULL
  AND resultcd <> ''
  AND resultcd <> ' '
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t2_temp a
LEFT JOIN (SELECT DISTINCT kunnr, kunnr_hash FROM adt_work.hash_info) b
       ON a.kunnr = b.kunnr
GROUP BY a.kunnr

)

SELECT a.*, b.resultcd
  FROM t2 a
LEFT JOIN (SELECT * FROM t1) b
       ON a.kunnr = b.kunnr
      AND a.final_dialdate = b.dialdate
''')
load_to_athena(conn, tm, 'default.tm_upsell_result_table')

<< Delete s3://adt-sandbox/_create/tables/default.tm_upsell_result_table
...tmp_db name = tmp_20201208_13_26_35_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
Start creating table
drop table if exists default.tmp_20201208_13_26_35_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
<< table not exist

        create  table         default.tmp_20201208_13_26_35_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2_temp AS (
SELECT DISTINCT kunnr, resultcd, dialdate
  FROM t1
WHERE resultcd IS NOT NULL
  AND resultcd <> ''
  AND resultcd <> ' '
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t2_tem

0

## TM 거부 이력 고객 

In [11]:
conn = athena_connection()
tm_refuse = load_from_athena(conn, '''
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE resultcd = 'SE01'
''')

...tmp_db name = tmp_20201208_13_28_36_8598cb085908b38eb145b9f85b970a5a314660e2
Start creating table
drop table if exists default.tmp_20201208_13_28_36_8598cb085908b38eb145b9f85b970a5a314660e2
<< table not exist

        create  table         default.tmp_20201208_13_28_36_8598cb085908b38eb145b9f85b970a5a314660e2
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE resultcd = 'SE01'

..
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 5)
Elapsed Time = 21.06
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/2c0f16a3-c8aa-45c6-8d45-7eec398f2ca8 /home/skinet/tmp/tmp_20201208_13_28_36_8598cb085908b38eb145b9f85b970a5a314660e2 --recursive
Elapsed Time = 

## 1차 결과테이블 추출 (1번 ~ 15번 조건 반영)

In [12]:
df = load_from_athena(conn,'''
WITH pre_magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
  FROM default.monthly_deadline_{} a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
)

, magam AS (
SELECT *
	 , ROW_NUMBER() OVER (PARTITION BY kunnr ORDER BY conn_type ASC, ctnum DESC, aedat DESC, aezet DESC) AS kunnr_rownum
  FROM pre_magam
)

-- 이 하단부터 조건들

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE ctype_l IN ('U00', 'U10', 'U20', 'U30', 'U40', 'U50', 'R00', 'W00')
                                       OR ctype_m IN ('L07')
                                       OR ctype_s IN ('W05', 'W06')
)

, t3 AS (
SELECT '3' AS filter, *  FROM magam WHERE conn_type NOT IN ('A01', 'A02')
)

, t4 AS (
SELECT '4' AS filter, *  FROM magam WHERE itype_l IN ('2', '3') OR itype_m = '22' OR itype_s IN ('160', '217', '222', '223')
)

, t5 AS (
SELECT '5' AS filter, *  FROM magam WHERE conn_type = 'A02' AND itype_m = '21'
)

, t6 AS (
SELECT '6' AS filter, *  FROM magam WHERE nsok IS NOT NULL
)

, t7 AS (
SELECT '7' AS filter, *  FROM magam WHERE gros_code > '0'
)

, t8 AS (
SELECT '8' AS filter, *  FROM magam WHERE gross_type = 'Y'
)

, t9 AS (
SELECT '9' AS filter, * FROM magam WHERE adt1_yn = 'N'
)

, t10 AS (
SELECT '10' AS filter, * FROM magam WHERE tm_yn IS NULL OR tm_yn = '' OR tm_yn = ' ' OR tm_yn = 'N'
)

, t11 AS (
SELECT '11' AS filter, * FROM magam WHERE ctnum IN (SELECT DISTINCT ctnum FROM default.tmupsell_exclude) -- 별도 테이블 작성 쿼리 참조 
)

, t13 AS (
SELECT '13' AS filter, * FROM magam WHERE (itype_l = '1' AND (monexp_amt < 30000 OR monexp_amt > 400000))
                                       OR (itype_l = '4' AND (monexp_amt < 30000 OR monexp_amt > 150000))
)

, t15 AS (
SELECT '15' AS filter, * FROM magam WHERE kunnr_rownum > 1
)

, total AS (
SELECT * FROM t1 UNION ALL 
SELECT * FROM t2 UNION ALL 
SELECT * FROM t3 UNION ALL 
SELECT * FROM t4 UNION ALL 
SELECT * FROM t5 UNION ALL 
SELECT * FROM t6 UNION ALL 
SELECT * FROM t7 UNION ALL 
SELECT * FROM t8 UNION ALL
SELECT * FROM t9 UNION ALL
SELECT * FROM t10 UNION ALL
SELECT * FROM t11 UNION ALL
SELECT * FROM t13 UNION ALL
SELECT * FROM t15 
)

, final_filter AS (
SELECT ctnum, MIN(filter) AS filter FROM total WHERE filter IS NOT NULL AND filter <> '' AND filter <> ' ' GROUP BY ctnum 
)

SELECT b.filter, a.* FROM magam a LEFT JOIN final_filter b ON a.ctnum = b.ctnum 
'''.format(ym))

...tmp_db name = tmp_20201208_13_29_09_e6026ad9564c05ea471e74f08d8590eb76fd84f5
Start creating table
drop table if exists default.tmp_20201208_13_29_09_e6026ad9564c05ea471e74f08d8590eb76fd84f5
<< table not exist

        create  table         default.tmp_20201208_13_29_09_e6026ad9564c05ea471e74f08d8590eb76fd84f5
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH pre_magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
  FROM default.monthly_deadline_202011 a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
)

, magam AS (
SELECT *
	 , ROW_NUMBER() OVER (PARTITION BY kunnr ORDER BY conn_type ASC, ctnum DESC, aedat DESC, aezet DESC) AS kunnr_rownum
  FROM pre_magam
)

-- 이 하단부터 조건들

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE ctype_l IN ('U00', 'U10', 'U20', 'U30', 'U40', 'U50', 'R00', 'W00')
                                       OR ctype_m IN ('L

In [13]:
df[df['filter'].isnull()==True].tm_yn.value_counts(dropna=False)

Y    202531
Name: tm_yn, dtype: int64

## 이 위가 15번 고객번호 중복 제거까지 반영한 내용. 이 아래부터 16번 ~ 23번 조건.

In [14]:
# 불필요한 컬럼 제거 
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type', 'kunnr_rownum']
df.drop(drop_col, axis=1, inplace=True)

In [15]:
# 혹시 몰라서 카피떠놓기
df_copy = df.copy()

In [16]:
# df = df_copy.copy()

In [17]:
# 필터링하지 않을 이상전화번호 리스트 (이상한 전화번호지만 db에 남겨놓고 TM센터에서 수기로 다른 번호 찾기 시도)
no_delete = [
'00','01','02','03','010','0101','01111','010000','010226','0100000','0101111','0105256','0109152','0109364','01000000','01001638','02000000','010000000','010010010','010111111'
,'011111111','020000000','021111111','0100000000','0100000001','0100100000','0101010101','0101111111','0101111234','0101231234','0101234567','0110000000','0111110000','0111234567','0200000000'
,'0201021021','0202222222','0111111111','0211000000','0211111112','0212341234','0211112222','0222222222','0510000000','01000000000','01000000001','01000000250','01000000602','01000001111','01000009999'
,'01000010001','01000090000','01000112111','01001000000','01001000900','01001010101','01001111111','01010000000','01010001000','01011110000','01011111018','01011111110','01011111111'
,'01001001010','01011111112','01011111118','01011111234','01011111313','01011112222','01012341234','01012345678','01022222222','01100000000','01101111111','01101234567','01111111111'
,'01111112222','01111113333','01112311231','03200000000','05100000000','05100011111','05101111111','05101112222','010000000000','010111111111','011111111111','0100000000000','01000000000000'
,'010000000000000','0','1','2','3','10','101','1111','10000','10226','100000','101111','105256','109152','109364','1000000','1001638','2000000','10000000','10010010','10111111','11111111'
,'20000000','21111111','100000000','100000001','100100000','101010101','101111111','101111234','101231234','101234567','110000000','111110000','111234567','200000000','201021021','202222222'
,'111111111','211000000','211111112','212341234','211112222','222222222','510000000','1000000000','1000000001','1000000250','1000000602','1000001111','1000009999','1000010001','1000090000'
,'1000112111','1001000000','1001000900','1001010101','1001111111','1010000000','1010001000','1011110000','1011111018','1011111110','1011111111','1001001010','1011111112','1011111118','1011111234'
,'1011111313','1011112222','1012341234','1012345678','1022222222','1100000000','1101111111','1101234567','1111111111','1111112222','1111113333','1112311231','3200000000','5100000000','5100011111'
,'5101111111','5101112222','10000000000','10111111111','11111111111','100000000000','1000000000000','10000000000000']

In [18]:
# 이상휴대폰번호 판단 함수 
# 등차수열이거나(ex.01012345678) 다 똑같은 번호거나(ex.01000000000), 위의 이상번호 리스트에 포함되어 있거나.

def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if (all(answer) == True) or (str(x) in (no_delete)):
            return False
      else:
            return True     

In [19]:
# 16. 휴대폰번호 전처리 및 중복 제거

df['telf2'] = df['telf2'].str.replace('\D|\s', '', regex=True)  # 숫자가 아닌 글자랑 공백 날리기
df['telf2'] = np.where(df['telf2'].str[0] != '0', "0"+df['telf2'], df['telf2'])   # 맨 앞에 0이 없으면 0 붙여주기

# df['telf_valid_1'] = df['telf2'].str.contains('^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$', regex=True)  # 자릿수가 맞는지 확인하기
df['telf_valid_1'] = True # 자릿수가 맞는지 확인하기
df['telf_valid_2'] = df['telf2'].apply(valid_number)                                                        # 위에 정의한 함수를 통해 valid한지 확인하기
df['telf_valid_final'] = np.where((df['telf_valid_1'] == True) & (df['telf_valid_2'] == True), True, False)
df['telf_valid_final'] = df['telf_valid_final'].map({True : 1, False : 0})                                  # 최종적으로 valid한지 확인
    
df = df.sort_values(by=['conn_type', 'ctnum', 'aedat', 'aezet'], axis=0, ascending=[True, False, False, False])  # 정렬해서 중복된 거 제거                                 

df.loc[(df['filter'].isnull()==True) & (df[(df['filter'].isnull()==True) & ((df['telf_valid_final']==1))].duplicated(subset='telf2', keep='first'))
           , 'filter'] = 16

In [20]:
#16-2 직원 휴대폰번호 제거 
conn = athena_connection()
emp = load_from_athena(conn,
'''
SELECT DISTINCT *
  FROM sap.zsdt0070
''')

...tmp_db name = tmp_20201208_13_30_21_c7be5de2d456dcd103d0eda043e4a8500924d23d
Start creating table
drop table if exists default.tmp_20201208_13_30_21_c7be5de2d456dcd103d0eda043e4a8500924d23d
<< table not exist

        create  table         default.tmp_20201208_13_30_21_c7be5de2d456dcd103d0eda043e4a8500924d23d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT *
  FROM sap.zsdt0070

.
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 6)
Elapsed Time = 13.45
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/8556dd49-465d-400c-ad4f-7fd2d1a1a4da /home/skinet/tmp/tmp_20201208_13_30_21_c7be5de2d456dcd103d0eda043e4a8500924d23d --recursive
Elapsed Time = 7.94
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/8556dd49-465d-400c-ad4f-7fd2d1a1a4da
Removing tmp_dir!
Elapsed Time for downloading and loading = 6.08
Total Elapsed Time = 27.47


In [21]:
emp['tel_hand'] = emp['tel_hand'].str.replace('\D|\s', '', regex=True)  # 숫자가 아닌 글자랑 공백 날리기
emp['tel_hand'] = np.where(emp['tel_hand'].str[0] != '0', "0"+emp['tel_hand'], emp['tel_hand'])   # 맨 앞에 0이 없으면 0 붙여주기
df.loc[(df['filter'].isnull()==True) & (df['telf2'].isin(emp['tel_hand']))  , 'filter'] = 16.5

In [22]:
# 17. 주소 중복내역 제거

df = df.sort_values(by=['conn_type', 'ctnum', 'aedat', 'aezet'], axis=0, ascending=[True, False, False, False])  # 정렬해서 중복된 거 제거                                 
df.loc[(df['filter'].isnull()==True) & (df.duplicated(subset=['gad_add01', 'gad_add02'], keep='first')), 'filter'] = 17

In [23]:
# 18. 최근 1년 이내 민원접수 고객
conn = athena_connection()
voc = load_from_athena(conn, '''
SELECT DISTINCT b.ctnum, a.receiptdate, a.receipttime
  FROM telepro.tb_voc a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash 
 WHERE a.receiptdate >= \'{}\'
'''.format(ymd_minus_1year))

...tmp_db name = tmp_20201208_13_30_52_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
Start creating table
drop table if exists default.tmp_20201208_13_30_52_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
<< table not exist

        create  table         default.tmp_20201208_13_30_52_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT b.ctnum, a.receiptdate, a.receipttime
  FROM telepro.tb_voc a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash 
 WHERE a.receiptdate >= '20191101'

.......................
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 6)
Elapsed Time = 156.24
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/de56e7d0-4b66-43ab-8db6-7b42b19c12db /home/skinet/tmp/tmp_20201208_13_30_52_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d --recursive
Elapsed Time =

In [24]:
df.loc[(df['filter'].isnull()==True) & (df['ctnum'].isin(voc['ctnum'])) , 'filter'] = 18

In [25]:
# 19. 해약 문의
conn = athena_connection()
churn = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.s25csq, b.gubun AS cd, a.talkenddate, '이관내역 기준' AS info
  FROM telepro.tb_contact a
LEFT JOIN telepro.tb_trans_contact b
       ON a.contactid = b.contactid
 WHERE b.gubun IN ('0025', '0026')
   AND a.talkenddate >= \'{}\'
   
UNION ALL 

SELECT DISTINCT s25csq, contactmiddlecd AS cd, talkenddate, '상담유형(중) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactmiddlecd IN ('J50J53', 'K50K56', 'J50J54')
   AND talkenddate >= \'{}\'
   
UNION ALL

SELECT DISTINCT s25csq, contactminorcd AS cd, talkenddate, '상담유형(소) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactminorcd IN ('B50B51B83', 'A01A08A25', 'G50G52G64')
   AND talkenddate >= \'{}\'
)

SELECT b.ctnum, a.cd, a.talkenddate, a.info
  FROM t1 a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
'''.format(ymd_minus_6month_slash, ymd_minus_6month_slash, ymd_minus_6month_slash))

...tmp_db name = tmp_20201208_13_33_43_9828052866bcce722b2bca5dfa793507bb49430b
Start creating table
drop table if exists default.tmp_20201208_13_33_43_9828052866bcce722b2bca5dfa793507bb49430b
<< table not exist

        create  table         default.tmp_20201208_13_33_43_9828052866bcce722b2bca5dfa793507bb49430b
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT a.s25csq, b.gubun AS cd, a.talkenddate, '이관내역 기준' AS info
  FROM telepro.tb_contact a
LEFT JOIN telepro.tb_trans_contact b
       ON a.contactid = b.contactid
 WHERE b.gubun IN ('0025', '0026')
   AND a.talkenddate >= '2020/05/01'
   
UNION ALL 

SELECT DISTINCT s25csq, contactmiddlecd AS cd, talkenddate, '상담유형(중) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactmiddlecd IN ('J50J53', 'K50K56', 'J50J54')
   AND talkenddate >= '2020/05/01'
   
UNION ALL

SELECT DISTINCT s25csq, contactminorcd AS cd, talkenddate, '상담유형(소) 기준' AS info
  FROM telepro.tb_contact
 WHERE con

In [26]:
df.loc[(df['filter'].isnull()==True) & (df['ctnum'].isin(churn['ctnum'])) , 'filter'] = 19

In [27]:
# 20. tm 거부
df.loc[(df['filter'].isnull()==True) & df['kunnr'].isin(tm_refuse['kunnr']), 'filter'] = 20

In [28]:
# 21. 두낫콜
donotcall = pd.DataFrame()
donotcall['휴대폰번호'] = df['telf2'].drop_duplicates(keep='first')
donotcall = donotcall.sort_values('휴대폰번호')
donotcall.index.name = 'index'
donotcall.to_csv('donotcall_list.csv', encoding='utf-8-sig')

### ↑ 여기에서 반환된 donotcall_list.csv 파일을 두낫콜에 넣고, 뱉어진 파일을 업로드한 후 파일명을 아래 filename 변수에 담기

In [29]:
filename = '2058121711_20201202171301.csv'

donotcall = pd.read_csv(filename)
donotcall['Result'] = donotcall['Result'].str.replace('\D|\s', '', regex=True)

df.loc[(df['filter'].isnull()==True) & (~df['telf2'].isin(donotcall['Result'])), 'filter'] = 21

In [30]:
# 22. 최종접촉일 3개월 이내
df.loc[(df['filter'].isnull()==True) & (df['final_dialdate'].str[0:10] >= final_call_boundary), 'filter'] = 22

In [31]:
# 23. 가입불가 필터링 (상호, 주소 등)

filter1 = ~df['gad_name'].str.contains('\(사\)')
filter2 = ~df['gad_name'].str.contains('사단')
filter3 = ~df['gad_name'].str.contains('\(재\)')
filter4 = ~df['gad_name'].str.contains('재단')

filter5 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('건물')) | (df['jongmok'].str.contains('건물'))))
filter6 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('대지')) | (df['jongmok'].str.contains('대지'))))
filter7 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('부동산')) | (df['jongmok'].str.contains('부동산'))))
filter8 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('빌딩')) | (df['jongmok'].str.contains('빌딩'))))
filter9 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('임대')) | (df['jongmok'].str.contains('임대'))))
filter10 = ~((df['gad_name'].str.contains('건물')) & ((df['uptae'].str.contains('점포')) | (df['jongmok'].str.contains('점포'))))

filter11 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('건물')) | (df['jongmok'].str.contains('건물'))))
filter12 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter13 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('부동산')) | (df['jongmok'].str.contains('부동산'))))
filter14 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('시설')) | (df['jongmok'].str.contains('시설'))))
filter15 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('임대')) | (df['jongmok'].str.contains('임대'))))
filter16 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('주차장')) | (df['jongmok'].str.contains('주차장'))))
filter17 = ~((df['gad_name'].str.contains('관리')) & ((df['uptae'].str.contains('주택')) | (df['jongmok'].str.contains('주택'))))

filter18 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter19 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('건축')) | (df['jongmok'].str.contains('건축'))))
filter20 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('관리')) | (df['jongmok'].str.contains('관리'))))
filter21 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('부동산')) | (df['jongmok'].str.contains('부동산'))))
filter22 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('상가')) | (df['jongmok'].str.contains('상가'))))
filter23 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('임대')) | (df['jongmok'].str.contains('임대'))))
filter24 = ~((df['gad_name'].str.contains('빌딩')) & ((df['uptae'].str.contains('주택')) | (df['jongmok'].str.contains('주택'))))

filter25 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter26 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('관리')) | (df['jongmok'].str.contains('관리'))))
filter27 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('부동산')) | (df['jongmok'].str.contains('부동산'))))
filter28 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('아파트')) | (df['jongmok'].str.contains('아파트'))))
filter29 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('임대')) | (df['jongmok'].str.contains('임대'))))
filter30 = ~((df['gad_name'].str.contains('빌라')) & ((df['uptae'].str.contains('주택')) | (df['jongmok'].str.contains('주택'))))

filter31 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('관리')) | (df['jongmok'].str.contains('관리'))))
filter32 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('번영')) | (df['jongmok'].str.contains('번영'))))
filter33 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('부동산')) | (df['jongmok'].str.contains('부동산'))))
filter34 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('상가')) | (df['jongmok'].str.contains('상가'))))
filter35 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('임대')) | (df['jongmok'].str.contains('임대'))))
filter36 = ~((df['gad_name'].str.contains('상가')) & ((df['uptae'].str.contains('주차')) | (df['jongmok'].str.contains('주차'))))

filter37 = ~((df['gad_name'].str.contains('아파트')) & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter38 = ~((df['gad_name'].str.contains('아파트')) & ((df['uptae'].str.contains('건축')) | (df['jongmok'].str.contains('건축'))))

filter39 = ~((df['gad_name'].str.contains('양식장')) & (df['itype_m'] != '02'))

filter40 = ~df['gad_name'].str.contains('원룸')

filter41 = ~((df['gad_name'].str.contains('조합')) & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter42 = ~((df['gad_name'].str.contains('조합')) & ((df['uptae'].str.contains('재건축')) | (df['jongmok'].str.contains('재건축'))))
filter43 = ~((df['gad_name'].str.contains('조합')) & ((df['uptae'].str.contains('주택')) | (df['jongmok'].str.contains('주택'))))

filter44 = ~df['gad_name'].str.contains('주차장')
filter45 = ~df['gad_name'].str.contains('타이어 뱅크')
filter46 = ~df['gad_name'].str.contains('타이어뱅크')
filter47 = ~df['gad_name'].str.contains('노인정')
filter48 = ~df['gad_name'].str.contains('기숙사')

filter49 = ~((df['gad_name'].str.contains('농장')) & (df['itype_m'] != '02') & (~(df['uptae'].str.contains('음식')) & (~df['jongmok'].str.contains('음식'))))

filter50 = ~df['gad_name'].str.contains('보험')
filter51 = ~df['gad_name'].str.contains('복지')
filter52 = ~df['gad_name'].str.contains('게스트하우스')
filter53 = ~df['gad_name'].str.contains('펜션')
filter54 = ~df['gad_name'].str.contains('고시원')
filter55 = ~df['gad_name'].str.contains('고시텔')

filter56 = ~((df['uptae'].str.contains('복지')) | (df['jongmok'].str.contains('복지')))
filter57 = ~((df['uptae'].str.contains('민박')) | (df['jongmok'].str.contains('민박')))
filter58 = ~((df['uptae'].str.contains('여관')) | (df['jongmok'].str.contains('여관')))

filter59 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('숙박')) | (df['jongmok'].str.contains('숙박'))) & (~(df['uptae'].str.contains('음식')) & (~df['jongmok'].str.contains('음식'))))
filter60 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('히우스')) | (df['jongmok'].str.contains('하우스')))) 
filter61 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('호텔')) | (df['jongmok'].str.contains('호텔'))) & (~(df['uptae'].str.contains('비품')) & (~df['jongmok'].str.contains('비품'))))
filter62 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('호텔')) | (df['jongmok'].str.contains('호텔'))) & (~(df['uptae'].str.contains('애견')) & (~df['jongmok'].str.contains('애견'))))
filter63 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('호텔')) | (df['jongmok'].str.contains('호텔'))) & (~(df['uptae'].str.contains('용품')) & (~df['jongmok'].str.contains('용품'))))


filter64 = ~((df['itype_l'] != '4') & ((df['uptae'].str.contains('다가구')) | (df['jongmok'].str.contains('다가구'))))
filter65 = ~((df['itype_l'] != '4') & ((df['uptae'].str.contains('빌딩')) | (df['jongmok'].str.contains('빌딩'))))
filter66 = ~((df['itype_l'] != '4') & ((df['uptae'].str.contains('빌라')) | (df['jongmok'].str.contains('빌라'))))
filter67 = ~((df['itype_l'] != '4') & ((df['uptae'].str.contains('아파트')) | (df['jongmok'].str.contains('아파트'))))

filter68 = ~((df['uptae'].str.contains('가축')) | (df['jongmok'].str.contains('가축')))
filter69 = ~((df['uptae'].str.contains('게스트하우스')) | (df['jongmok'].str.contains('게스트하우스')))
filter70 = ~((df['uptae'].str.contains('고시원')) | (df['jongmok'].str.contains('고시원')))
filter71 = ~((df['uptae'].str.contains('고시텔')) | (df['jongmok'].str.contains('고시텔')))
filter72 = ~((df['uptae'].str.contains('과수원')) | (df['jongmok'].str.contains('과수원')))
filter73 = ~((df['uptae'].str.contains('관공서')) | (df['jongmok'].str.contains('관공서')))
filter74 = ~((df['uptae'].str.contains('국가기관')) | (df['jongmok'].str.contains('국가기관')))
filter75 = ~((df['uptae'].str.contains('글램핑')) | (df['jongmok'].str.contains('글램핑')))

filter76 = ~((df['uptae'] == '글램핑') | (df['jongmok'] == '글램핑'))

filter77 = ~((df['uptae'].str.contains('농원')) | (df['jongmok'].str.contains('농원')))
filter78 = ~((df['uptae'].str.contains('농장')) | (df['jongmok'].str.contains('농장')))
filter79 = ~((df['uptae'].str.contains('돈사')) | (df['jongmok'].str.contains('돈사')))
filter80 = ~((df['uptae'].str.contains('묘목')) | (df['jongmok'].str.contains('묘목')))
filter81 = ~((df['uptae'].str.contains('보험')) | (df['jongmok'].str.contains('보험')))
filter82 = ~((df['uptae'].str.contains('비영리')) | (df['jongmok'].str.contains('비영리')))
filter83 = ~((df['uptae'].str.contains('생화')) | (df['jongmok'].str.contains('생화')))
filter84 = ~((df['uptae'].str.contains('손해사정')) | (df['jongmok'].str.contains('손해사정')))
filter85 = ~((df['uptae'].str.contains('식물')) | (df['jongmok'].str.contains('식물')))
filter86 = ~((df['uptae'].str.contains('양계')) | (df['jongmok'].str.contains('양계')))
filter87 = ~((df['uptae'].str.contains('양봉')) | (df['jongmok'].str.contains('양봉')))

filter88 = ~((df['itype_m'] != '02') & ((df['uptae'].str.contains('양식')) | (df['jongmok'].str.contains('양식'))) & (~(df['uptae'].str.contains('음식')) & (~df['jongmok'].str.contains('음식'))))

filter89 = ~((df['uptae'].str.contains('양어장')) | (df['jongmok'].str.contains('양어장')))
filter90 = ~((df['uptae'].str.contains('어업')) | (df['jongmok'].str.contains('어업')))
filter91 = ~((df['uptae'].str.contains('원룸')) | (df['jongmok'].str.contains('원룸')))
filter92 = ~((df['uptae'].str.contains('조경')) | (df['jongmok'].str.contains('조경')))
filter93 = ~((df['uptae'].str.contains('축사')) | (df['jongmok'].str.contains('축사')))

filter94 = ~(((df['uptae'].str.contains('캠핑')) | (df['jongmok'].str.contains('캠핑'))) & (~(df['uptae'].str.contains('용품')) & (~df['jongmok'].str.contains('용품'))))
filter95 = ~(((df['uptae'].str.contains('캠핑')) | (df['jongmok'].str.contains('캠핑'))) & (~(df['uptae'].str.contains('장비')) & (~df['jongmok'].str.contains('장비'))))

filter96 = ~((df['uptae'].str.contains('펜션')) | (df['jongmok'].str.contains('펜션')))
filter97 = ~((df['uptae'].str.contains('화훼')) | (df['jongmok'].str.contains('화훼')))

filter98 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('화원') & (df['itype_m'] != '02')))
filter99 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('화원') & (df['itype_m'] != '06')))

filter100 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('단란주점')))
filter101 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('유흥주점')))
filter102 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('단란주점')) | (df['jongmok'].str.contains('단란주점'))))
filter103 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('유흥주점')) | (df['jongmok'].str.contains('유흥주점'))))
filter104 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('노래')))
filter105 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('노래')) | (df['jongmok'].str.contains('노래'))))

filter106 = ~((df['conn_type'] == 'A02') & ((df['uptae'] == '도매') | (df['jongmok'] == '도매')))
filter107 = ~((df['conn_type'] == 'A02') & ((df['uptae'] == '도매업') | (df['jongmok'] == '도매업')))

filter108 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('창고')) | (df['jongmok'].str.contains('창고'))))
filter109 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('중개인')))
filter110 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('가구')) | (df['jongmok'].str.contains('가구'))))

filter111 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('가구')))
filter112 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('테크')))
filter113 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('정밀')))
filter114 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('건재')))
filter115 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('물류')))
filter116 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('철강')))
filter117 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('자원')))
filter118 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('고물')))
filter119 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('기계')))
filter120 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('정공')))
filter121 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('석재')))
filter122 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('연마')))
filter123 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('금형')))
filter124 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('공장')))
filter125 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('창고')))
filter126 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('제조')))
filter127 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('요양원')))
filter128 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('리싸이클')))
filter129 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('사우나')))
filter130 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('찜질')))
filter131 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('목욕')))
filter132 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('폐차')))
filter133 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('스텐')))
filter134 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('산업')))
filter135 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('금속')))
filter136 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('무역')))
filter137 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('철재')))
filter138 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('가설')))
filter139 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('공업사')))
filter140 = ~((df['conn_type'] == 'A02') & (df['gad_name'].str.contains('택배')))

filter141 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('무역')) | (df['jongmok'].str.contains('무역'))))
filter142 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('공장')) | (df['jongmok'].str.contains('공장'))))
filter143 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('제조')) | (df['jongmok'].str.contains('제조'))))
filter144 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('철근')) | (df['jongmok'].str.contains('철근'))))
filter145 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('사우나')) | (df['jongmok'].str.contains('사우나'))))
filter146 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('찜질')) | (df['jongmok'].str.contains('찜질'))))
filter147 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('목욕')) | (df['jongmok'].str.contains('목욕'))))
filter148 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('목재')) | (df['jongmok'].str.contains('목재'))))
filter149 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('연구')) | (df['jongmok'].str.contains('연구'))))
filter150 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('폐차')) | (df['jongmok'].str.contains('폐차'))))
filter151 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('고물')) | (df['jongmok'].str.contains('고물'))))
filter152 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('재활용')) | (df['jongmok'].str.contains('재활용'))))
filter153 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('고철')) | (df['jongmok'].str.contains('고철'))))
filter154 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('기계')) | (df['jongmok'].str.contains('기계'))))
filter155 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('농기구')) | (df['jongmok'].str.contains('농기구'))))
filter156 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('모델하우스')) | (df['jongmok'].str.contains('모델하우스'))))
filter157 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('사격')) | (df['jongmok'].str.contains('사격'))))
filter158 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('파지')) | (df['jongmok'].str.contains('파지'))))
filter159 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('비철')) | (df['jongmok'].str.contains('비철'))))
filter160 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('제생영')) | (df['jongmok'].str.contains('제생영'))))
filter161 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('산업')) | (df['jongmok'].str.contains('산업'))))
filter162 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('가공')) | (df['jongmok'].str.contains('가공'))))

filter163 = ~((df['conn_type'] == 'A02') & (df['itype_m'] != '05') & ((df['uptae'].str.contains('장비')) | (df['jongmok'].str.contains('장비'))) & ((df['uptae'] == '소매') | (df['jongmok'] == '소매')))

filter164 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('태양광')) | (df['jongmok'].str.contains('태양광'))))
filter165 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('절')) | (df['jongmok'].str.contains('절'))))
filter166 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('법당')) | (df['jongmok'].str.contains('법당'))))
filter167 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('불교')) | (df['jongmok'].str.contains('불교'))))
filter168 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('사찰')) | (df['jongmok'].str.contains('사찰'))))
filter169 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('용접')) | (df['jongmok'].str.contains('용접'))))
filter170 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('대중탕')) | (df['jongmok'].str.contains('대중탕'))))
filter171 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('인쇄')) | (df['jongmok'].str.contains('인쇄'))))
filter172 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('폐기물')) | (df['jongmok'].str.contains('폐기물'))))
filter173 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('토목')) | (df['jongmok'].str.contains('토목'))))

filter174 = ~((df['conn_type'] == 'A02') & (df['itype_m'] != '05') & ((df['uptae'].str.contains('전자상거래')) | (df['jongmok'].str.contains('전자상거래'))) & ((df['uptae'] == '소매') | (df['jongmok'] == '소매')) & (df['gad_add01'].str.contains('[^1]층', regex=True)))

filter175 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('종자')) | (df['jongmok'].str.contains('종자'))))
filter176 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('반도체')) | (df['jongmok'].str.contains('반도체'))))
filter177 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('석재')) | (df['jongmok'].str.contains('석재'))))
filter178 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('건설')) | (df['jongmok'].str.contains('건설'))))
filter179 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('광업')) | (df['jongmok'].str.contains('광업'))))
filter180 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('분양')) | (df['jongmok'].str.contains('분양'))))
filter181 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('자재')) | (df['jongmok'].str.contains('자재'))))
filter182 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('합판')) | (df['jongmok'].str.contains('합판'))))
filter183 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('요양원')) | (df['jongmok'].str.contains('요양원'))))
filter184 = ~((df['conn_type'] == 'A02') & ((df['uptae'].str.contains('구내식당')) | (df['jongmok'].str.contains('구내식당'))))

filter185 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('지하')) | (df['gad_add02'].str.contains('지하'))))
filter186 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('지층')) | (df['gad_add02'].str.contains('지층'))))
filter187 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('[Bb]\d', regex=True)) | (df['gad_add02'].str.contains('[Bb]\d', regex=True))))
filter188 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('도매시장')) | (df['gad_add02'].str.contains('도매시장'))))
filter189 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('어시장')) | (df['gad_add02'].str.contains('어시장'))))
filter190 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('산물시장')) | (df['gad_add02'].str.contains('산물시장'))))
filter191 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('종합시장')) | (df['gad_add02'].str.contains('종합시장'))))
filter192 = ~((df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('수산시장')) | (df['gad_add02'].str.contains('산물시장'))))

filter193 = ~((df['conn_type'] == 'A02') & (df['itype_s'] == '062') & ((df['gad_add01'].str.contains('[^1]층', regex=True)) | (df['gad_add02'].str.contains('[^1]층', regex=True))))

filter194 = ~((df['itype_l'] == '4') & (df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('[^1]층', regex=True)) | (df['gad_add02'].str.contains('[^1]층', regex=True))))
filter195 = ~((df['itype_l'] == '4') & (df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('[^1]호', regex=True)) | (df['gad_add02'].str.contains('[^1]호', regex=True))))
filter196 = ~((df['itype_l'] == '4') & (df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('빌라') | (df['gad_add02'].str.contains('빌라')))))  
filter197 = ~((df['itype_l'] == '4') & (df['conn_type'] == 'A02') & ((df['gad_add01'].str.contains('아파트') | (df['gad_add02'].str.contains('아파트')))))                

In [32]:
df_temp = df.copy()

In [33]:
# df = df_temp.copy()

In [34]:
df.shape

(773429, 52)

In [35]:
# 필터 돌리기
for i in range(1, 197):
      df = df[globals()['filter' + str(i)]]

In [36]:
df.shape

(679250, 52)

In [37]:
df = pd.DataFrame(df['ctnum'].astype(str))
df['name_fine'] = 1
df = df_temp.merge(df, how='left', on='ctnum')
df = df.sort_values(by=['conn_type', 'ctnum', 'aedat', 'aezet'], axis=0, ascending=[True, False, False, False])
df = df.drop_duplicates(subset='ctnum', keep='first')
df.shape

(773427, 53)

In [38]:
df.loc[(df['filter'].isnull()==True) & (df['name_fine'].isnull()==True), 'filter'] = 23

In [39]:
df['filter'].fillna(0).astype(int).value_counts(dropna=False).sort_index()

0     146448
1       2880
2       1202
3         99
4       8478
5       7213
6       1745
7       2089
8        210
9        960
10    400680
11     46772
13     32516
15     66052
16     11468
17      7025
18      3330
19      4747
20     11928
21      3528
23     14057
Name: filter, dtype: int64

In [41]:
df[df['filter'].isnull()==True].to_csv('sample.csv', encoding='utf-8-sig')

In [42]:
df['filter'].fillna(0).astype(int).value_counts(dropna=False).sort_index().to_csv('전체db.csv', encoding='utf-8-sig')

In [43]:
df[df['filter'].isnull()==True]['con_cnt'].value_counts(dropna=False).sort_index().to_csv('temp.csv', encoding='utf-8-sig')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 6))

df_sort = df[df['filter'].isnull()==True].con_cnt.value_counts().sort_index()

for i in df_sort.index:
    ax.annotate(f"{df_sort[i]}", 
                   xy=(i-1, df_sort[i]+500), va='center', ha='center',fontweight='light', fontfamily='serif', color='#4a4a4a')

for s in ['top', 'left', 'right']:
    ax.spines[s].set_visible(False)

sns.countplot(x=df[df['filter'].isnull()==True].con_cnt, order=df[df['filter'].isnull()==True].con_cnt.value_counts().sort_index().index, 
             edgecolor='darkgray', color='#d4dddd', linewidth=0.7)

ax.set_xticklabels(df_sort.index.astype(int), fontfamily='serif')
ax.set(xlabel="Contact Count ", ylabel = "DB Count")
plt.ylim(0, 20000)

In [ ]:
df.drop(['ctnum_hash', 'kunnr_hash', 'name_fine', 'telf_valid_1', 'telf_valid_2', 'telf_valid_final'], axis=1, inplace=True)

df['filter'].value_counts()

In [ ]:
df = df.rename(columns={
      'gubun' : '구분',
      'filter' : '필터',
      'kunnr' : '고객번호',
      'ctnum' : '계약번호',
      'cstat' : '계약상태',
      'gad_name' : '상호',
      'gad_start' : '경비개시일',
      'conn_start' : '계약개시일',
      'conn_end' : '계약종료일',
      'monexp_amt' : '월정료',
      'conn_type' : '계약타입',
      'ctype_l' : '채널(대)',
      'ctype_m' : '채널(중)',
      'ctype_s' : '채널(소)',
      'itype_l' : '업종(대)',
      'itype_m' : '업종(중)',
      'itype_s' : '업종(소)',
      'name1' : '이름1',
      'gad_add01' : '경비물건주소',
      'post_add01' : '우편물수령지주소1',
      'first_bildate' : '최초청구일',
      'telf2' : '전화번호',
      'line_type' : '관제회선구분',
      'uptae' : '업태',
      'jongmok' : '종목',
      'gros_code' : '그로스코드',
      'gross_type' : '그로스타입',
      'kvgr4' : '세그먼트',
      'gad_pstlz' : '우편번호',
      'gad_add02' : '경비물건주소상세',
      'post_add02' : '우편물수령지주소2',
      'sms_yn' : 'SMS동의',
      'email_yn' : 'EMAIL동의',
      'adt1_yn' : '개인정보수집이용동의',
      'tm_yn' : 'TM동의',
      'dm_yn' : 'DM동의',
      'sk1_yn' : 'SK1동의',
      'sk2_yn' : 'SK2동의',
      'aedat' : '작업일자',
      'aezet' : '작업시간',
      'aenam' : '변경자',
      'nsok' : 'n사계약번호',
      'final_dialdate' : '최종접촉일시',
      'resultcd' : '최종접촉결과',
      'con_cnt' : '접촉회차'
})

In [ ]:
df = df[
    ['필터', '고객번호', '계약번호', '계약상태', '상호', '경비개시일', '계약개시일', '계약종료일', 
     '월정료', '계약타입', '채널(대)', '채널(중)', '채널(소)', '업종(대)', '업종(중)', '업종(소)', 
     '이름1', '경비물건주소', '우편물수령지주소1', '최초청구일', '전화번호',
     '관제회선구분', '업태', '종목', '그로스코드', '그로스타입', '세그먼트', '우편번호', '경비물건주소상세',
     '우편물수령지주소2', '개인정보수집이용동의', 'SMS동의', 'EMAIL동의', 'TM동의', 'DM동의', 'SK1동의', 'SK2동의', 
     '작업일자', '작업시간', '변경자', 'n사계약번호', '최종접촉일시', '최종접촉결과', '접촉회차']]

In [ ]:
df.to_csv('상세.csv', encoding='utf-8-sig')

In [ ]:
workdt = datetime.datetime.now()
workdt = str(workdt.year) + str(workdt.month).zfill(2) + str(workdt.day).zfill(2)
gizon_month = datetime.date(int(first_bill_month[0:4]), int(first_bill_month[4:6]), 1) - relativedelta(months=1)
gizon_month = str(gizon_month.year)+str(gizon_month.month).zfill(2)

filename = gizon_month[2:4] +'년'+ gizon_month[4:6] + '월기존_' + first_bill_month[2:4] +'년'+ first_bill_month[4:6] + '월신규_TM_UPSELL_DB_' + workdt + '.csv'
filename

In [ ]:
df.to_csv(filename, encoding='utf-8-sig')